### Train Models for CIFAR 10  

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

In [2]:
import os
import sys
import time
import math

import torch.nn.init as init

In [3]:
# vgg_cifar100.py

from utils import * 
from cifar10_architecture.densenet_cifar100 import * 

In [4]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | TRAIN Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [5]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | TEST Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/densenet161_cifar100.pth')
        best_acc = acc

In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [7]:
trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [9]:
print('==> Building model..')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("Device: {}".format(device))

net = densenet161()
net = net.to(device)

if device == 'cuda:0':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

==> Building model..
Device: cuda:0


In [10]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
 [================================================================>]  Step: 225ms | Tot: 2m10s | Loss: 3.629 | TRAIN Acc: 14.366% (7183/5000 391/391 1 
 [================================================================>]  Step: 82ms | Tot: 8s251ms | Loss: 3.293 | TEST Acc: 20.360% (2036/1000 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 225ms | Tot: 2m11s | Loss: 2.721 | TRAIN Acc: 30.212% (15106/5000 391/391 1 
 [================================================================>]  Step: 82ms | Tot: 8s249ms | Loss: 2.754 | TEST Acc: 30.980% (3098/1000 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 224ms | Tot: 2m10s | Loss: 2.189 | TRAIN Acc: 41.154% (20577/5000 391/391 1 
 [================================================================>]  Step: 82ms | Tot: 8s211ms | Loss: 2.333 | TEST Acc: 40.640% (4064/1000 100/100 
Saving..

Epoch: 3
 [=======================

 [================================================================>]  Step: 83ms | Tot: 8s294ms | Loss: 1.260 | TEST Acc: 71.130% (7113/1000 100/100 

Epoch: 51
 [================================================================>]  Step: 227ms | Tot: 2m11s | Loss: 0.119 | TRAIN Acc: 96.546% (48273/5000 391/391 1  
 [================================================================>]  Step: 85ms | Tot: 8s319ms | Loss: 1.287 | TEST Acc: 70.900% (7090/1000 100/100 

Epoch: 52
 [================================================================>]  Step: 227ms | Tot: 2m11s | Loss: 0.113 | TRAIN Acc: 96.788% (48394/5000 391/391 1  
 [================================================================>]  Step: 82ms | Tot: 8s303ms | Loss: 1.319 | TEST Acc: 69.980% (6998/1000 100/100 

Epoch: 53
 [================================================================>]  Step: 226ms | Tot: 2m11s | Loss: 0.106 | TRAIN Acc: 97.036% (48518/5000 391/391 1  
 [=====================================================

 [================================================================>]  Step: 82ms | Tot: 8s229ms | Loss: 1.199 | TEST Acc: 73.710% (7371/1000 100/100 

Epoch: 102
 [================================================================>]  Step: 224ms | Tot: 2m10s | Loss: 0.036 | TRAIN Acc: 99.146% (49573/5000 391/391 1  
 [================================================================>]  Step: 84ms | Tot: 8s207ms | Loss: 1.194 | TEST Acc: 74.110% (7411/1000 100/100 

Epoch: 103
 [================================================================>]  Step: 227ms | Tot: 2m10s | Loss: 0.026 | TRAIN Acc: 99.436% (49718/5000 391/391 1  
 [================================================================>]  Step: 81ms | Tot: 8s201ms | Loss: 1.150 | TEST Acc: 74.690% (7469/1000 100/100 
Saving..

Epoch: 104
 [================================================================>]  Step: 229ms | Tot: 2m10s | Loss: 0.024 | TRAIN Acc: 99.458% (49729/5000 391/391 1  
 [=========================================

 [================================================================>]  Step: 79ms | Tot: 8s90ms | Loss: 0.993 | TEST Acc: 78.800% (7880/1000 100/100 
Saving..

Epoch: 153
 [================================================================>]  Step: 225ms | Tot: 2m10s | Loss: 0.005 | TRAIN Acc: 99.970% (49985/5000 391/391 1  
 [================================================================>]  Step: 81ms | Tot: 8s209ms | Loss: 0.989 | TEST Acc: 78.780% (7878/1000 100/100 

Epoch: 154
 [================================================================>]  Step: 224ms | Tot: 2m10s | Loss: 0.005 | TRAIN Acc: 99.978% (49989/5000 391/391 1  
 [================================================================>]  Step: 81ms | Tot: 8s170ms | Loss: 0.989 | TEST Acc: 78.740% (7874/1000 100/100 

Epoch: 155
 [================================================================>]  Step: 224ms | Tot: 2m10s | Loss: 0.005 | TRAIN Acc: 99.984% (49992/5000 391/391 1  
 [==========================================

In [4]:
torch.load("checkpoint/densenet161_cifar100.pth", map_location='cuda:0')['acc']

79.05